In [1]:
import os
import torch
from cqt_loader import *
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
#import models
from config import opt
from torchnet import meter
from tqdm import tqdm
import numpy as np
import time
import torch.nn.functional as F
import torch
import torch.nn as nn
from utility import *
#from models.CQTNet import CQTNet
from models import CQTNet

/home/rodrigo/Documents/CQTNet/cqt_loader.py:92: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.mode is 'train':


In [2]:
parallel = True 
opt.model = 'CQTNet'
opt.notes='CQTNet'
opt.batch_size=32
#opt.load_latest=True
#opt.load_model_path = ''
#opt._parse(kwargs)

### step1: configure model

In [3]:
model = getattr(CQTNet, opt.model)()
if parallel is True: 
    model = torch.nn.DataParallel(model)
if parallel is True:
    if opt.load_latest is True:
        model.module.load_latest(opt.notes)
    elif opt.load_model_path:
        model.module.load(opt.load_model_path)
else:
    if opt.load_latest is True:
        model.load_latest(opt.notes)
    elif opt.load_model_path:
        model.load(opt.load_model_path)
model.to(opt.device)
print(model)


DataParallel(
  (module): CQTNet(
    (features): Sequential(
      (conv0): Conv2d(1, 32, kernel_size=(12, 3), stride=(1, 1), padding=(6, 0), bias=False)
      (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (conv1): Conv2d(32, 64, kernel_size=(13, 3), stride=(1, 1), dilation=(1, 2), bias=False)
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
      (conv2): Conv2d(64, 64, kernel_size=(13, 3), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(1, 2), bias=False)
      (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

/home/rodrigo/anaconda3/envs/env/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


### step2: data

In [4]:
train_data0 = CQT('train', out_length=200)
train_data1 = CQT('train', out_length=300)
train_data2 = CQT('train', out_length=400)
val_data350 = CQT('songs350', out_length=None)
val_data80 = CQT('songs80', out_length=None)
val_data = CQT('val', out_length=None)
test_data = CQT('test', out_length=None)
val_datatMazurkas = CQT('Mazurkas', out_length=None)
train_dataloader0 = DataLoader(train_data0, opt.batch_size, shuffle=True,num_workers=opt.num_workers)
train_dataloader1 = DataLoader(train_data1, opt.batch_size, shuffle=True,num_workers=opt.num_workers)
train_dataloader2 = DataLoader(train_data2, opt.batch_size, shuffle=True,num_workers=opt.num_workers)
val_dataloader = DataLoader(val_data, 1, shuffle=False,num_workers=1)
test_dataloader = DataLoader(test_data, 1, shuffle=False,num_workers=1)
val_dataloader80 = DataLoader(val_data80, 1, shuffle=False, num_workers=1)
val_dataloader350 = DataLoader(val_data350, 1, shuffle=False, num_workers=1)
val_dataloaderMazurkas = DataLoader(val_datatMazurkas,1, shuffle=False,num_workers=1)

### step3: criterion and optimizer

In [5]:
criterion = torch.nn.CrossEntropyLoss()
lr = opt.lr
if parallel is True:
    optimizer = torch.optim.Adam(model.module.parameters(), lr=lr, weight_decay=opt.weight_decay)
else:
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=opt.weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,mode='min',factor=opt.lr_decay,patience=2, verbose=True,min_lr=5e-6)

/home/rodrigo/anaconda3/envs/env/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


### train

In [8]:
best_MAP=0
val_slow(model, val_dataloader350, -1)
for epoch in range(opt.max_epoch):
    running_loss = 0
    num = 0
    for (data0, label0),(data1, label1),(data2, label2) in tqdm(zip(train_dataloader0, train_dataloader1, train_dataloader2)):
        for flag in range(3):
            if flag==0:
                data=data0
                label=label0
            elif flag==1:
                data=data1
                label=label1
            else:
                data=data2
                label=label2
            # train model
            input = data.requires_grad_()
            input = input.to(opt.device)
            target = label.to(opt.device)

            optimizer.zero_grad()
            score, _ = model(input)
            loss = criterion(score, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            num += target.shape[0]
    running_loss /= num 
    print(running_loss)
    if parallel is True:
        model.module.save(opt.notes)
    else:
        model.save(opt.notes)
    # update learning rate
    scheduler.step(running_loss) 
    # validate
    MAP=0
    MAP += val_slow(model, val_dataloader350, epoch)
    MAP += val_slow(model, val_dataloader80, epoch)
    val_slow(model, val_dataloaderMazurkas, epoch)
    if MAP>best_MAP:
        best_MAP=MAP
        print('*****************BEST*****************')
    print('')
    model.train()



set_id:0	0	Summertime	Abbie Mitchell	https://www.youtube.com/watch?v=TMHWEcSKEPI	True




set_id:0	1	Summertime	Helen Jepson	https://www.youtube.com/watch?v=E3lw8gaHs04	True




set_id:0	2	Summertime	Billie Holiday and Her Orchestra	https://www.youtube.com/watch?v=aPpJPTJc1lU	True




ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/rodrigo/anaconda3/envs/env/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/rodrigo/anaconda3/envs/env/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/rodrigo/anaconda3/envs/env/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/home/rodrigo/Documents/CQTNet/cqt_loader.py", line 87, in __getitem__
    set_id, version_id = filename.split('.')[0].split('_')
    ^^^^^^^^^^^^^^^^^^
ValueError: not enough values to unpack (expected 2, got 1)


In [9]:
@torch.no_grad()
def val_slow(model, dataloader, epoch):
    model.eval()
    total, correct = 0, 0
    labels, features = None, None

    for ii, (data, label) in enumerate(dataloader):
        input = data.to(opt.device)
        #print(input.shape)
        score, feature = model(input)
        feature = feature.data.cpu().numpy()
        label = label.data.cpu().numpy()
        if features is not None:
            features = np.concatenate((features, feature), axis=0)
            labels = np.concatenate((labels,label))
        else:
            features = feature
            labels = label
    features = norm(features)
    #dis2d = get_dis2d4(features)
    dis2d = -np.matmul(features, features.T) # [-1,1] Because normalized, so mutmul is equal to ED
    np.save('dis80.npy',dis2d)
    np.save('label80.npy',labels)
    if len(labels) == 350:
        MAP, top10, rank1 = calc_MAP(dis2d, labels,[100, 350])
    #elif len(labels) == 160:    MAP, top10, rank1 = calc_MAP(dis2d, labels,[80, 160])
    else :
        MAP, top10, rank1 = calc_MAP(dis2d, labels)

    print(epoch, MAP, top10, rank1 )
    model.train()
    return MAP
